# Kompresja bezstratna - wstęp i kodowanie Huffmana

In [232]:
import bitarray as b
def create(signs_with_frequencies_sorted):
    #założono, że jest 37 znaków, wiec tylko przypadek z 6 bitami jest rozpatrywany
    length_of_code_in_bits = 0
    if len(signs_with_frequencies_sorted)>=32 and len(signs_with_frequencies_sorted)<=64:
        length_of_code_in_bits = 6
    
    if length_of_code_in_bits == 0:
        print("Zbyt dużo znaków dla tego algorytmu")
        return 0
    else:
        code_words_dict = {}
        counter = 0
        for sign in signs_with_frequencies_sorted:
            tmp = b.bitarray(bin(counter)[2:])
            for i in range(0,5):
                tmp.insert(0,False)
            code_words_dict[sign] = tmp[-6:]
            counter += 1
        return code_words_dict

In [233]:
def createHuffman(signs_with_frequencies_sorted):
    helper = []
    for i in range(0,len(signs_with_frequencies_sorted)):
        tmp = signs_with_frequencies_sorted[i]
        tmp.append("")
        helper.append(tmp)
        
    while len(helper) != 1:
        base = ""
        tmp = helper[0][2].split(" ")
        for i in tmp:
            base += i+"0 "
        tmp1 = helper[1][2].split(" ")
        for i in range(0, len(tmp1)-1):
            base += tmp1[i]+"1 "
        base += tmp1[-1]+"1"
        helper[0] = [helper[0][0]+","+helper[1][0],helper[0][1]+helper[1][1],base]
        helper.pop(1)
        pom = sorted(helper, key=lambda l: l[1])
        helper = pom.copy()
    
    code = {}
    signs = helper[0][0].split(",")
    base_codes = helper[0][2].split(" ")
    for sign, base_code in zip(signs,base_codes):
        tmp = base_code[::-1]
        code[sign] = b.bitarray(tmp)
    return code

In [234]:
def encode(code,text):
    encoded = b.bitarray(0)
    for sign in text:
        encoded += code[sign]
    return encoded

In [235]:
def decode(code,encoded):
    text = ""
    tmp = b.bitarray(0)
    for i in encoded:
        if i:
            tmp = tmp + b.bitarray('1')
        else:
            tmp = tmp + b.bitarray('0')
        try:
            sign = list(code.keys())[list(code.values()).index(tmp)]
            tmp = b.bitarray(0)
            text += sign
        except:
            continue
            
    return text

In [246]:
def load(filenameCode, filename):
    code = {}
    with open(filenameCode, 'r') as file:
        lines = file.readlines()
    for line in lines:
        code[line[0]] = b.bitarray(line[2:-1]) 
        
    with open(filename, 'rb') as file:
        encoded = b.bitarray(0)
        encoded.fromfile(file)
    return [code, encoded]

In [247]:
def save(code,encoded,filenameCode,filename):
    with open(filenameCode, 'w') as file:
        for code_word in code.items():
            line = code_word[0] + " " + code_word[1].to01()+"\n"
            file.write(line)
    with open(filename, 'wb') as file:
        encoded.tofile(file)

In [248]:
import string

signs = list(string.ascii_lowercase)
signs.append(" ")
for i in range(0,10):
    signs.append(str(i))
    
with open('dane1.txt', 'r') as file:
    data = file.read()

how_many = [0 for i in range(0, len(signs))]
for sign in data:
    how_many[signs.index(sign)] += 1   

result = []
for sign, count in zip(signs, how_many):
    result.append([sign, count])
sorted_result = sorted(result, key=lambda l: l[1], reverse=True)

signs_with_frequencies_sorted = []
for e in sorted_result:
    signs_with_frequencies_sorted.append(e[0])

    
code = create(signs_with_frequencies_sorted)
encoded = encode(code,data)
save(code,encoded,"kompresjaBezstratnaKod.txt","kompresjaBezstratna.txt")
res = load("kompresjaBezstratnaKod.txt","kompresjaBezstratna.txt")
decoded = decode(res[0],res[1])
print(res[0])
if data == decoded:
    print("OK")
    print(decoded[0:200])

{' ': bitarray('000000'), 'e': bitarray('000001'), 'a': bitarray('000010'), 't': bitarray('000011'), 'i': bitarray('000100'), 'n': bitarray('000101'), 'o': bitarray('000110'), 'r': bitarray('000111'), 's': bitarray('001000'), 'h': bitarray('001001'), 'l': bitarray('001010'), 'd': bitarray('001011'), 'c': bitarray('001100'), 'm': bitarray('001101'), 'u': bitarray('001110'), 'f': bitarray('001111'), 'p': bitarray('010000'), 'g': bitarray('010001'), 'b': bitarray('010010'), 'w': bitarray('010011'), 'y': bitarray('010100'), 'v': bitarray('010101'), 'k': bitarray('010110'), '1': bitarray('010111'), '0': bitarray('011000'), '9': bitarray('011001'), '2': bitarray('011010'), 'j': bitarray('011011'), '8': bitarray('011100'), 'x': bitarray('011101'), '3': bitarray('011110'), '5': bitarray('011111'), '4': bitarray('100000'), '6': bitarray('100001'), '7': bitarray('100010'), 'z': bitarray('100011'), 'q': bitarray('100100')}
OK
 albert of prussia 17 may 1490 20 march 1568 was the last grand master 

In [249]:
result = []
for sign, count in zip(signs, how_many):
    probability = count / len(data)
    result.append([sign, probability])
sorted_result = sorted(result, key=lambda l: l[1])

code_huffman = createHuffman(sorted_result)
print(code_huffman)
encoded = encode(code_huffman,data)
save(code_huffman,encoded,"kompresjaBezstratnaHuffmanKod.txt","kompresjaBezstratnaHuffman.txt")
res = load("kompresjaBezstratnaHuffmanKod.txt","kompresjaBezstratnaHuffman.txt")
decoded = decode(res[0],res[1])
if data == decoded:
    print("OK")
    print(decoded[0:200])

{'e': bitarray('000'), 'm': bitarray('00100'), 'k': bitarray('0010100'), '4': bitarray('001010100'), '5': bitarray('001010101'), '2': bitarray('00101011'), 'y': bitarray('001011'), 's': bitarray('0011'), 'w': bitarray('010000'), 'b': bitarray('010001'), 'c': bitarray('01001'), 'r': bitarray('0101'), 'o': bitarray('0110'), 'n': bitarray('0111'), 'i': bitarray('1000'), '9': bitarray('10010000'), '3': bitarray('100100010'), 'x': bitarray('100100011'), '8': bitarray('100100100'), 'j': bitarray('100100101'), '0': bitarray('10010011'), 'g': bitarray('100101'), 'd': bitarray('10011'), 't': bitarray('1010'), 'l': bitarray('10110'), 'p': bitarray('101110'), 'f': bitarray('101111'), 'a': bitarray('1100'), 'h': bitarray('11010'), 'v': bitarray('1101100'), 'q': bitarray('1101101000'), 'z': bitarray('1101101001'), '7': bitarray('1101101010'), '6': bitarray('1101101011'), '1': bitarray('11011011'), 'u': bitarray('110111'), ' ': bitarray('111')}
OK
 albert of prussia 17 may 1490 20 march 1568 was the

In [254]:
import math
sredniaDlugosc = 0
for i in code_huffman.items():
    sredniaDlugosc += len(i[1].to01())
sredniaDlugosc = sredniaDlugosc/37
entropia = 0
for i in sorted_result:
    entropia += i[1] * math.log(1/i[1],2)
efektywnoscHuffman = entropia/ sredniaDlugosc
efektywnoscHuffman = efektywnoscHuffman * 100
print("entropia: "+str(entropia)+"bit")
print("efektywność kodowania Huffmana: "+str(efektywnoscHuffman)+"%")

entropia: 4.266132006831655bit
efektywność kodowania Huffmana: 65.49663247002955%
